This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

##### First I will create python virtual environment using command `python -m venv env` and in jupyter notebook in VScode, I select this virtual environment as my main kernel
##### Note: I have to use chromedriver-binary version `123.0.6312.86.0` for the compability
##### Second note: My OS is Windows 11

In [1]:
%pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
%pip install selenium chromedriver-binary==123.0.6312.86.0


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: d:\Data_major\icttm_submit\env\Scripts\python.exe -m pip install --upgrade pip



  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/12/88/86ae378d3db210eb91a6d02c31f52690bf11510665dd9519d960024c0359/boto3-1.34.72-py3-none-any.whl.metadata
  Using cached boto3-1.34.72-py3-none-any.whl.metadata (6.6 kB)
  Obtaining dependency information for botocore<1.35.0,>=1.34.72 from https://files.pythonhosted.org/packages/dc/7e/23b72fa8190bb16c40d7ae8ca287eb5334e3977ddd31d57ce3f49a2a108b/botocore-1.34.72-py3-none-any.whl.metadata
  Using cached botocore-1.34.72-py3-none-any.whl.metadata (5.7 kB)
  Obtaining dependency information for jmespath<2.0.0,>=0.7.1 from https://files.pythonhosted.org/packages/31/b4/b9b800c45527aadd64d5b442f9b932b00648617eb5d63d2c7a6587b7cafc/jmespath-1.0.1-py3-none-any.whl.metadata
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/83/37/395cdb6ee92925fa211e55d8f07b9f93cf93f60d7d4ce5e66fd73f


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: d:\Data_major\icttm_submit\env\Scripts\python.exe -m pip install --upgrade pip


In [1]:
import os
import boto3

#### Using selenium to download file

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import chromedriver_binary

# set download directory
download_dir = r'D:\Data_major\icttm_submit'
prefs = {"download.default_directory": download_dir}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", prefs)

# initialize the Chrome WebDriver with options
driver = webdriver.Chrome(options=chrome_options)

driver.get('https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC?usp=sharing')
time.sleep(15) # add delay to wait for finishing loading page

# find data.zip file and download button using xpath
file_element = driver.find_element(By.XPATH, "//div[contains(@aria-label, 'Lưu trữ nén: data.zip')]")
file_element.click()

download_button = driver.find_element(By.XPATH, "//div[contains(@data-tooltip, 'Tải xuống')]")
download_button.click()

action = ActionChains(driver)
action.context_click(file_element).perform()
action.send_keys(Keys.ARROW_DOWN).perform()
action.send_keys(Keys.ENTER).perform()

# Close the browser
driver.quit()

#### Unzip to folder `data`

In [5]:
import zipfile

# define the path to the zip file and the directory to extract to
zip_file_path = 'D:\Data_major\icttm_submit\data.zip'
extract_dir = 'D:\Data_major\icttm_submit\data'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


In [6]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'i8UnD6H3PyxR0UFEuUY7')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', '8JFkDk4otgUQd1EOlKIfQIQPq8EzxqEBSzBz8dda')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')


In [7]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)


In [8]:
# Upload file to minio
upload_file_to_minio('data/data.json', 'data', 'data-raw/data.json')
upload_file_to_minio('data/data2.json', 'data', 'data-raw/data2.json')
upload_file_to_minio('data/data3.json', 'data', 'data-raw/data3.json')